In [1]:
import geopandas as gpd
import pandas as pd
from shapely.ops import nearest_points

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_placas = gpd.read_file('resultados/sinalizacao-pompeia-vila-leopoldina.gpkg')
gdf_points = gpd.read_file('resultados/points-pompeia-vila-leopoldina.gpkg')

In [3]:
gdf_distritos = gpd.read_file('zip://downloads/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito', crs='EPSG:31983')
gdf_distritos = gdf_distritos.set_crs(epsg=31983)

In [4]:
gdf_placas.loc[:, ['tema', 'informacao', 'grupo']] = [[p0, p1, p2] for [p0, p1, p2] in gdf_placas.value.str.split('--')]

In [5]:
gdf_placas.tema.value_counts()

regulatory       7952
information      6863
warning           540
complementary     233
general            52
Name: tema, dtype: int64

In [6]:
len(gdf_placas.informacao.value_counts())

127

In [7]:
gdf_placas.grupo.value_counts()

g1     10175
g5      2209
g2      1840
g3      1116
g6       130
g4        90
g9        78
g8         1
g10        1
Name: grupo, dtype: int64

In [8]:
gdf_placas[gdf_placas.informacao.str.contains('parking')].to_file('resultados/no-parking-pompeia-vila-leopoldina.gpkg', driver='GPKG', crs=4326)

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [9]:
gdf_points[gdf_points.value.str.contains('walk')].to_file('resultados/walking-pompeia-vila-leopoldina.gpkg', driver='GPKG', crs=4326)

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [10]:
gdf_points['value'].value_counts()

object--support--utility-pole                        17195
object--street-light                                  8630
object--sign--advertisement                           4529
object--sign--store                                   3620
marking--discrete--crosswalk-zebra                    1974
construction--flat--driveway                          1952
object--traffic-light--general-upright                1615
object--support--pole                                 1515
object--traffic-sign--front                           1262
object--trash-can                                      911
object--traffic-sign--direction-front                  756
object--manhole                                        660
marking--discrete--other-marking                       641
object--traffic-cone                                   554
object--traffic-light--pedestrians                     531
object--junction-box                                   450
marking--discrete--stop-line                           4

In [12]:
convex_hull = gdf_placas.to_crs(epsg=31983).dissolve().geometry.convex_hull

## TRabalhando com o distrito da Lapa

In [13]:
gdf_distritos = gdf_distritos[gdf_distritos.ds_nome == 'LAPA']
gdf_distritos

,ds_cd_sub,ds_sigla,ds_subpref,ds_nome,ds_codigo,geometry
68,08,LAP,LAPA,LAPA,48,"POLYGON ((323721.814 7398433.381, 323730.119 7..."


In [14]:
gdf_placas = gdf_placas.to_crs(epsg=31983).overlay(gdf_distritos[gdf_distritos.ds_nome == 'LAPA'])

In [15]:
dist = gdf_distritos.iloc[0]
gdf_meios_fios = gpd.read_file(f'../poligono-de-vias/resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg')


In [16]:
gdf_placas.sjoin_nearest(gdf_meios_fios, distance_col='distancia_original')

,first_seen_at,id,last_seen_at,value,tema,informacao,grupo,ds_cd_sub,ds_sigla,ds_subpref,ds_nome,ds_codigo,geometry,index_right,lg_codlog,lg_seg_id,lado_esquerdo,distancia_original
0,2019-04-11T15:34:56,183956490173832,2019-04-11T15:34:56,regulatory--maximum-speed-limit-50--g1,regulatory,maximum-speed-limit-50,g1,08,LAP,LAPA,LAPA,48,POINT (324197.353 7398685.270),1726,124362,45138.0,True,1.685264
117,2018-07-27T20:13:39,183051416931006,2018-07-27T20:13:39,regulatory--maximum-speed-limit-60--g1,regulatory,maximum-speed-limit-60,g1,08,LAP,LAPA,LAPA,48,POINT (324193.563 7398681.410),1726,124362,45138.0,True,0.798345
614,2018-07-27T20:13:39,184077823495032,2018-07-27T20:13:39,information--general-directions--g1,information,general-directions,g1,08,LAP,LAPA,LAPA,48,POINT (324203.398 7398683.708),1726,124362,45138.0,True,7.530673
1,2018-09-01T15:39:18,1373485323007659,2018-09-01T15:39:18,regulatory--no-parking--g5,regulatory,no-parking,g5,08,LAP,LAPA,LAPA,48,POINT (324727.559 7398424.116),373,029289,45168.0,True,1.327614
2,2019-05-28T12:22:20,498666581577656,2019-05-28T12:22:20,information--general-directions--g1,information,general-directions,g1,08,LAP,LAPA,LAPA,48,POINT (323955.039 7397967.033),957,060143,45534.0,False,0.300919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4587,2018-05-15T17:10:07,306515624273417,2018-05-15T17:10:07,regulatory--turn-right--g2,regulatory,turn-right,g2,08,LAP,LAPA,LAPA,48,POINT (327366.066 7396993.263),2463,190047,47083.0,False,4.874720
4594,2019-06-12T18:44:55,578158490240523,2019-06-12T18:44:55,regulatory--turn-left--g3,regulatory,turn-left,g3,08,LAP,LAPA,LAPA,48,POINT (327746.027 7397339.294),726,050725,49718.0,True,3.448408
4609,2018-05-14T17:54:00,897219200839050,2018-05-14T17:54:00,regulatory--no-parking--g5,regulatory,no-parking,g5,08,LAP,LAPA,LAPA,48,POINT (327780.046 7397334.791),1301,076880,50572.0,False,5.968986
4612,2021-03-14T13:22:20,344792330418539,2021-03-14T13:22:20,information--general-directions--g1,information,general-directions,g1,08,LAP,LAPA,LAPA,48,POINT (327762.415 7397250.147),796,053180,50906.0,True,16.311679


In [17]:
gdf_placas_reposicionadas = gdf_placas.sjoin_nearest(gdf_meios_fios, distance_col='distancia_original')

In [18]:
gdf_placas_reposicionadas.geometry = [nearest_points(r.geometry, gdf_meios_fios.loc[r.index_right].geometry)[-1] for (i, r) in gdf_placas_reposicionadas.iterrows()]

In [19]:
gdf_placas_reposicionadas.to_file('tmp/placas_reposicionadas.gpkg', driver='GPKG')

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
